## Lesson 1

In [ ]:
!which python

In [ ]:
import torch

In [ ]:
torch.__version__

## Lesson 2
Start simple with the buildingblocks that will be needed later

Assume we have a one dimensional input and try to model it with a linear function ancored at the origin

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
# create som simple data
def load_data():
    x = [1.0, 2.0, 3.0, 4.0]
    y = [2.0, 4.0, 6.0, 8.0]
    return x,y

x,y = load_data()
plt.plot(x,y)

In [ ]:
w = 2

def forward(x):
    return x * w

# is it good practice to implement the forward pass in the acutall loss or should it be done outside?
def loss_simple(x, y):
    n = len(x)
    l = 0
    for x_in, y_in in zip(x, y):
        y_pred = forward(x_in)
        l += (y_pred - y_in)**2
    return l/n

def loss(x,y):
    y_forward = forward(x)
    return (y_forward - y)**2

Another super simple example only planting the seed of what ml is. Next lesson should cover backpropagation

## Lesson 3

In [ ]:
def gradient(x, y):
    return 2*x*(x*w-y)

eta = 0.1
def step(x,y):
    return w - eta * gradient(x,y)

mse_list = []
w_list = []
w = 5.0
for epoch in range(10):
    for x_val, y_val in zip(x,y):
        w = step(x_val,y_val)
    
    w_list.append(w)
    mse_list.append(loss(x_val, y_val))
    print(w)

## Lesson 4
Actually using the gradient calculations based in pytorch

In [ ]:
from torch.autograd import Variable
w = Variable(torch.Tensor([1.0]), requires_grad=True)

def foward(x):
    return w * x

def loss(x,y):
    y_pred = forward(x)
    return (y_pred - y)**2

x,y = load_data()
print("before training:", 4, forward(4).data[0])
for epoch in range(10):
    for x_val, y_val in zip(x,y):
        l = loss(x_val, y_val)
        l.backward()
        w.data = w.data -  0.01 * w.grad.data
        print("\tgrad: ", x_val, y_val, w.grad.data[0])
        
        w.grad.data.zero_()
    print("Progress after epoch:", epoch, l.data[0])

print("after training:", 4, forward(4).data[0])

## Lesson 5
Linear regression

In [ ]:
def get_tensor_data():
    x = Variable(torch.Tensor([[1.0], [2.0], [4.0]]))
    y = Variable(torch.Tensor([[2.0], [4.0], [8.0]]))
    return x,y

x,y = get_tensor_data()

class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear = torch.nn.Linear(1, 1)
    
    
    def forward(self, x):
        return self.linear(x)

model = Model()

In [ ]:
# the criterion is a function that determines how the loss gradients can be calculated
criterion = torch.nn.MSELoss(size_average=False)

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

for epoch in range(500):
    y_pred = model.forward(x)
    
    loss = criterion(y_pred, y)
    #print(epoch, loss.data)
    
    #plt.scatter(epoch, loss.data)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(loss.item())

The workflow with tensorflow follow these steps:
1. Design model 
2. Select a criterion (loss function) and an optimizer. These will guide the model towards a minimum...
3. ... together with a training loop. This needs a forward pass, a loss calculation, gradiet reset, gradient calculation and weight update step. 



## Lesson 6 - Logistic regression


In [ ]:

torch.manual_seed(1)

def get_classifier_data():
    x,_ = get_tensor_data()
    y_data = Variable(torch.Tensor([[0.], [0.], [1.]]))

    return x, y_data

class LogisticClassifier(torch.nn.Module):
    def __init__(self):
        super(LogisticClassifier, self).__init__()
        self.linear = torch.nn.Linear(1,1)
        
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred
    
clf = LogisticClassifier()

In [ ]:
criterion = torch.nn.BCELoss(reduction="sum")
optimizer = torch.optim.Adam(clf.parameters(), lr=0.1)
x, y = get_classifier_data()

In [ ]:
for e in range(500):
    y_pred = clf.forward(x)
    loss = criterion(y_pred, y)
    
    optimizer.zero_grad()
    loss.backward()
    l = round(loss.item(), 4)
    plt.scatter(e, l)
    optimizer.step()
    
    print(f"Epoch:",  e, l )

Questions for next time:
1. What frameworks does already exist when it comes to creating and training networks. Basically how can i come around the fact that I in these scenarios always have to write the entire three step process from scratch


# Lesson 7


In [ ]:
import numpy as np
import torch
from torch.autograd import Variable

In [ ]:
xy = np.loadtxt('diabetes.csv', delimiter=';', dtype=np.float32)
x_data = Variable(torch.from_numpy(xy[:,:-1]))
y_data = Variable(torch.from_numpy(xy[:,[-1]]))

In [ ]:
y_data.shape[1]

In [ ]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

class DNN(torch.nn.Module):
    h1 = 6
    h2 = 4
    
    def __init__(self, x, y):
        super(DNN, self).__init__()
        
        self.wi = torch.nn.Linear(x.shape[1], 6)
        self.wh = torch.nn.Linear(self.h1, self.h2)
        self.wo = torch.nn.Linear(self.h2, y.shape[1])
        
        
    def forward(self, x):
        o = torch.relu(self.wi(x))
        o = torch.relu(self.wh(o))
        return torch.sigmoid(self.wo(o))

def train(x, y, model, epochs):
    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
    loss_history = []
    for epoch in range(epochs):
        y_pred = model(x)
        loss = criterion(y_pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_history.append((epoch, loss))
    return loss_history
model = DNN(x_data, y_data)
l = train(x_data, y_data, model, 100)

In [ ]:
from matplotlib import pyplot as plt
x = [i[0] for i in l]
y = [i[1] for i in l]
plt.plot(x, y)
plt.show()

## Lesson 8 - DataLoader
`DataLoader` makes it easier to batch data and maybe also to split it into test and train sets. This is super useful since it encapsulates the nitty-gritty stuff that otherwise might go wrong (and if they do, things are in trouble)

In [30]:
from torch.utils.data import Dataset, DataLoader
import time
class DNN(torch.nn.Module):
    h1 = 6
    h2 = 4
    
    def __init__(self):
        super(DNN, self).__init__()
        
        self.wi = torch.nn.Linear(8, 6)
        self.wh = torch.nn.Linear(self.h1, self.h2)
        self.wo = torch.nn.Linear(self.h2, 1)
        
        
    def forward(self, x):
        o = torch.relu(self.wi(x))
        o = torch.relu(self.wh(o))
        return torch.sigmoid(self.wo(o))

class DiabetesDataset(Dataset):
    
    def __init__(self):
        xy = np.loadtxt('diabetes.csv', delimiter=';', dtype=np.float32)

        self.x_data = Variable(torch.from_numpy(xy[:,:-1]))
        self.y_data = Variable(torch.from_numpy(xy[:,[-1]]))
        self.len = len(self.x_data)
    
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len

dataset = DiabetesDataset()

#Define size of test and train split
val_size = 168
train_size = len(dataset) - val_size

#Split data 
train, val = torch.utils.data.random_split(dataset, [train_size, val_size])

#Create data loaders for both sets
data_loader = DataLoader(train, batch_size=32, shuffle=True, num_workers=0, drop_last=False)
val_data_loader = DataLoader(val, batch_size=val_size, shuffle=False, num_workers=0, drop_last=False)

#Create model and define loss and optimizer
model = DNN()
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

#Training loop
history = []
for e in range(500):
    start = time.time()
    epoch_train_loss = []
    for i, (x, y) in enumerate(data_loader, 1):
        y_pred = model.forward(x)
        loss = criterion(y_pred, y)
        
        epoch_train_loss.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    #end for
    
    with torch.no_grad():
        epoch_val_loss = []
        for x_val, y_val in val_data_loader:
            
            val_pred = model.forward(x_val)
            l = criterion(val_pred, y_val)
            epoch_val_loss.append(l.item())
        #end for
    #end with
    train_loss = sum(epoch_train_loss)/len(epoch_train_loss)
    val_loss = sum(epoch_val_loss)/len(epoch_val_loss)
    history.append([e, train_loss, val_loss])
    if e % 10 == 9:
        print(f"Epoch: {e} took {round(time.time() - start, 3)} \t Train loss: {round(train_loss, 3)} \t Val loss: {round(val_loss, 3)}")

    

Epoch: 9 took 0.016 	 Train loss: 0.659 	 Val loss: 0.617
Epoch: 19 took 0.016 	 Train loss: 0.655 	 Val loss: 0.621
Epoch: 29 took 0.016 	 Train loss: 0.656 	 Val loss: 0.62
Epoch: 39 took 0.017 	 Train loss: 0.656 	 Val loss: 0.622
Epoch: 49 took 0.016 	 Train loss: 0.655 	 Val loss: 0.626
Epoch: 59 took 0.016 	 Train loss: 0.655 	 Val loss: 0.621
Epoch: 69 took 0.016 	 Train loss: 0.658 	 Val loss: 0.618
Epoch: 79 took 0.016 	 Train loss: 0.656 	 Val loss: 0.621
Epoch: 89 took 0.016 	 Train loss: 0.655 	 Val loss: 0.622
Epoch: 99 took 0.016 	 Train loss: 0.655 	 Val loss: 0.62
Epoch: 109 took 0.016 	 Train loss: 0.654 	 Val loss: 0.621
Epoch: 119 took 0.016 	 Train loss: 0.658 	 Val loss: 0.618
Epoch: 129 took 0.016 	 Train loss: 0.657 	 Val loss: 0.621
Epoch: 139 took 0.015 	 Train loss: 0.655 	 Val loss: 0.622
Epoch: 149 took 0.015 	 Train loss: 0.654 	 Val loss: 0.62
Epoch: 159 took 0.016 	 Train loss: 0.655 	 Val loss: 0.622
Epoch: 169 took 0.016 	 Train loss: 0.656 	 Val loss: 

RuntimeError: In set_text: Could not load glyph (error code 0x55)

In [ ]:
plt.plot(np.array([h[0] for h in history]), np.array([h[1] for h in history]))

In [ ]:
import time
import copy
dataset = DiabetesDataset()

#Define size of test and train split
val_size = 168
train_size = len(dataset) - val_size

#Split data 
train, val = torch.utils.data.random_split(dataset, [train_size, val_size])

#Create data loaders for both sets
data_loader = DataLoader(train, batch_size=32, shuffle=True, num_workers=2, drop_last=False)
val_data_loader = DataLoader(val, batch_size=val_size, shuffle=False, num_workers=2, drop_last=False)


model = DNN()
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.1)
train_model(model, {"train": data_loader, "val": val_data_loader}, criterion, optimizer, 10)